# Imports

The **services module** contains functions for accessing third party bioemdical data services.

In [1]:
from services.pdb import pfam, pdbligand, pdbmolecule
from services.classyfire import classyfire
from services.uniprot import pdb2uniprot
from services.uniprot import pdb2uniprotAC
from services.pubchem import get_image

##### PDB
For PDB we have functions for quering information about (1) PDB files, (2) Protein Molecules, (3) Ligands, (4) PFam Annotations and (5) GO annotations.  

##### Uniprot
For Uniprot we query a service that maps pdb IDs to uniprot IDs. 

# Pocket Feature Graph

In [2]:
import pandas as pd
import networkx as nx
import numpy as np

##### Step 1. Convert pocket feature scores to "cosine similarities"

Given a matrix $X \in \mathbb R ^n$ with rows $x_i$, we compute the cosine similarity between two rows as:

$$ \frac{x_i^T x_j}{\lVert x_i \rVert \lVert x_j \rVert }$$

Given symmetric positive semidefinite matrix $A = X X^T$, we can compute cosine similarities using hte formula:

$$\frac{a_{ij}}{\sqrt{a_{ii}}\sqrt{a_{jj}}} = \frac{x_i^T x_j}{\lVert x_i \rVert \lVert x_j \rVert }$$ 

In [3]:
# Read in pocket feature similarities and convert to an adjacency matrix
pf = pd.read_csv('./data/pocket_feature_scores.csv', header=None, names=['pocket_0', 'pocket_1', 'weight'])
pf_matrix = pf.pivot(index='pocket_0', columns='pocket_1', values='weight')

# Compute cosine similarities from pocket feature score matrix
diagonal = np.sqrt(np.diag(-1*pf_matrix))
denominator = np.outer(diagonal, diagonal)
normalized = (-1*pf_matrix)/denominator

FileNotFoundError: [Errno 2] File ./data/pocket_feature_scores.csv does not exist: './data/pocket_feature_scores.csv'

##### Step 2. Filter cosine similarities

In order to reduce memory and computational costs, we filter out cosine slimilarity values that fall outside of a range of interest.  For example, we may wish to exclude low similarity values.  Likewise, we do not wish to have self referencing edges in our graph, so we also filter out diagonal values in the adjacency matrix.

In [ ]:
# Specify desired range
score_range = [0.6, 0.85]
lower_limit = score_range[0]
upper_limit = score_range[1]

# Set matrix entries out of specificed range to NaN
if lower_limit > 0:
    normalized[normalized < lower_limit] = np.nan
if upper_limit < 1:
    normalized[normalized > upper_limit] = np.nan

# Set off Diagonal to NaN
np.fill_diagonal(normalized.values, np.nan)

##### Step 3. Initialize PF Similarity Graph

In [ ]:
# Convert from adjacency matrix to edge list
normalized_edges = normalized.stack().reset_index()
normalized_edges = normalized_edges.rename(columns={0:'weight'})

In [ ]:
# Split off ligand IDs from pocket IDs and add them as separate columns
new = normalized_edges['pocket_0'].str.split("_", expand = True)
normalized_edges['ligand_0'] = new[1]

new = normalized_edges['pocket_1'].str.split("_", expand = True)
normalized_edges['ligand_1'] = new[1]

In [ ]:
# Initialize graph
PFG = nx.Graph()

# Iterate through rows of edge list dataframe
for row in normalized_edges.itertuples():
    
    # If both ligand are the same, do nothing
    ligand_0 = row.ligand_0
    ligand_1 = row.ligand_1
    if ligand_0 == ligand_1:
        continue
        
    # If ligands are different, we pull out pocket IDs and pf cosine similarity
    pocket_0 = row.pocket_0    
    pocket_1 = row.pocket_1
    weight = row.weight
    
    # If we have not seen this pair of ligand before, we add an edge
    if not PFG.has_edge(ligand_0, ligand_1):
        PFG.add_edge(ligand_0, ligand_1, id=[pocket_0, pocket_1], weight=weight)
        
    # Otherise update the edge weight if pf cosine similarity is greater than previous max
    elif weight > PFG[ligand_0][ligand_1]['weight']:
        PFG[ligand_0][ligand_1]['id'] = [pocket_0, pocket_1]
        PFG[ligand_0][ligand_1]['weight'] = weight

# Scaffold Hop Scores

We define a scaffold hop score for a pair of ligands as:

$$max\left(\frac{F(m_1, m_2)}{T(m_1,m_2)}\right)$$

Where $m_i$ denotes a ligand in our dataset, $F$ is the pocket feature cosine similarity between bound protein structures, and $T$ is the tanomoto coefficient between chemical fingerprints.  Ligands may be bound in multiple co-crystal structures, and for any pair of ligands there may be a range of scaffold hop scores.  We take the maximum value.  In general, the high scaffold hop scores corrspond to ligands with dissimilar strucutres, that bind slimilar protein pockets.

In [ ]:
# Import ligand tanomoto matrix
ligand_matrix = pd.read_csv('./data/ligandComparisons.csv', index_col=0)

In [ ]:
SHG = PFG.copy()
for ligand_0, ligand_1 in PFG.edges:
    ligand_score = ligand_matrix[ligand_0][ligand_1]
    pf_score = PFG[ligand_0][ligand_1]['weight']
    
    SHG[ligand_0][ligand_1]['weight'] = pf_score/ligand_score

# Filtering

In [ ]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [ ]:
from networkx.algorithms import bipartite

def projected_graph(Graph):
    nx.set_node_attributes(Graph, bipartite.color(Graph), name='color')

    top = [i for i in Graph.nodes if Graph.nodes[i]['color'] == 1]
    projected = bipartite.overlap_weighted_projected_graph(Graph, top)
    return projected

all_pockets = []
for u,v in SHG.edges:
    pockets = SHG[u][v]['id']
    all_pockets.extend(pockets)
    
structures, ligands = zip( *[i.split('_') for i in set(all_pockets)] )
structures = list(set(structures))

pdb_uniprot_mapping = pd.read_csv(pdb2uniprot(structures), sep='\t')

PUG = nx.from_pandas_edgelist(pdb_uniprot_mapping, source='From', target='To')
uniprot_filter = projected_graph(PUG)

In [ ]:
filtered = SHG.copy()
for u,v in SHG.edges:
    try:
        pdb_0, pdb_1 = [i.split('_')[0] for i in filtered[u][v]['id']]
        if uniprot_filter[pdb_0][pdb_1]['weight'] >= 0.5:
            filtered.remove_edge(u,v)
    except KeyError:
        pass

In [ ]:
#Looking for affinities specific to 4J33
for ligand_0, ligand_1 in PFG.edges:
    pockets = SHG[ligand_0][ligand_1]['id']
    if (ligand_0 == 'FAD' or ligand_1 == 'FAD'): 
        ligand_score = ligand_matrix[ligand_0][ligand_1]
        pf_score = PFG[ligand_0][ligand_1]['weight']
        #print(pockets)
        #print(ligand_0)
        #print(ligand_1)
        #print(pf_score/ligand_score)
        
#Need to get edge data!

In [ ]:
#Looking for strongest edge to 4J33



In [ ]:
# Creates dict mapping pdbid to chain containing the binding pocket
header = {0:"pdb", 1:"lig", 2:"chain", 3:"num"}
pdbs = pd.read_csv('./data/PDB_List_Final.txt', header=None, sep='\t')
pdbs = pdbs.rename(columns=header)
pdb_dict = dict(zip(pdbs.pdb, pdbs.chain))

In [ ]:
from pprint import pprint
PDB_list = pdb_dict.keys()

count = 0

seqDict = {}
for mol in PDB_list:
    chain_entry = {}
    polymer = pdbmolecule(mol)['polymer']
    for j in range(len(polymer)): 
        if type(polymer) == list:
            if type(polymer[j]['chain']) == list: 
                chain = polymer[j]['chain'][0]['id']
            else:
                chain = polymer[j]['chain']['id']
            if 'macroMolecule' in polymer[j]: #Else uniprotACC is empty
                uniprotACC = polymer[j]['macroMolecule']['accession']['id']
            else:
                print('empty uniprot accession')
        else: 
            if type(polymer['chain']) == list: 
                chain = polymer['chain'][0]['id']
            else:
                chain = polymer['chain']['id']
            try:
                uniprotACC = polymer['macroMolecule']['accession']['id']
            except:
                print(mol)
                pprint(pdbmolecule(mol))
                pprint(polymer)
                print(len(polymer))
                count += 1
                print(f"whoopsie! Mistake number {count}")
                
        chain_entry[chain] = uniprotACC
        seqDict[mol] = chain_entry

In [ ]:
pdb_acc = {}
for pdbid in pdb_dict.keys():
    try: 
        pdb_acc[pdbid] = seqDict[pdbid][pdb_dict[pdbid]]
    except:
        missing = True

# Results

In [ ]:
from pprint import pprint

# pprint(sorted(SHG.edges(data=True), key=lambda x: x[2]['weight'], reverse=True)[:10])
# sorted_scores = sorted(SHG.edges(data=True), key=lambda x: x[2]['weight'], reverse=True)

pprint(sorted(filtered.edges(data=True), key=lambda x: x[2]['weight'], reverse=True)[:10])
sorted_scores = sorted(filtered.edges(data=True), key=lambda x: x[2]['weight'], reverse=True)

In [ ]:
import IPython.display as Disp

# image size can be 'large' or 'small'
image_size = 'large'

# Pick an index of example to inspect 
idx = 8

# everything below pulls info about example
edge = sorted_scores[idx]
lig_0 = edge[0]
lig_1 = edge[1]

lig_info = pdbligand( ','.join([lig_0, lig_1]) )
keys = ['chemicalID','chemicalName', 'formula', 'molecularWeight']
classes = []
for info in lig_info:
    inchi = info['InChIKey']
    co_classes = classyfire(inchi)
    classes.append(co_classes)
    txt = [co_classes['direct_parent']['name'], co_classes['description']]
    image = get_image(inchi, image_size=image_size)
#     url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{inchi}/PNG?image_size={image_size}' 
#     image = requests.get(url).content
    text = '<br>'.join( [info[key].upper() for key in keys] + txt )
    Disp.display( Disp.Image(image), Disp.HTML(text) )

In [ ]:
def extract_pdbids(edge_data):
    pdbs = edge_data['id']
    return ','.join([i.split('_')[0] for i in pdbs])

pprint(pdbmolecule(extract_pdbids(edge[2])))